# 第50章: 光学の基礎

## 📋 この章で学ぶこと

この章を終えると、以下ができるようになります：

- [ ] 光の性質（波動性・粒子性）と画像形成の関係を理解する
- [ ] 幾何光学の基本法則（反射・屈折）を説明できる
- [ ] レンズの結像原理と薄レンズの式を導出できる
- [ ] 焦点距離・F値・被写界深度の関係を理解する

## 🎯 前提知識

この章を学ぶには以下の知識が必要です：

- ✅ 高校物理（波動、光学）
- ✅ 基礎的な微積分

⏱️ **推定学習時間**: 120-150分  
📊 **難易度**: ★★☆☆☆（初級〜中級）  
🎓 **カテゴリ**: 理論 + 実践

---

## 🌟 はじめに

**3D Computer Vision**の基礎を学ぶ前に、まず「光」と「像の形成」の物理を理解しましょう。

カメラが3D世界を2D画像に変換する過程は、すべて**光学の法則**に従っています。この章では：

- なぜレンズが必要なのか
- どのように像が形成されるのか
- カメラのパラメータ（焦点距離、F値など）の物理的意味

を学びます。

### 🤔 なぜ光学を学ぶのか？

```
┌─────────────────────────────────────────────────────────┐
│                    3D Computer Vision                    │
├─────────────────────────────────────────────────────────┤
│                                                         │
│   物理世界 ──[光]──→ レンズ ──→ センサー ──→ 画像     │
│                                                         │
│   この変換過程を理解することが、                        │
│   3D復元・NeRF・Gaussian Splattingの基盤               │
│                                                         │
└─────────────────────────────────────────────────────────┘
```

In [ ]:
# ============================================================
# 環境設定
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Arc, FancyArrowPatch, Wedge, Circle
from matplotlib.lines import Line2D
import warnings

warnings.filterwarnings('ignore')

# グラフスタイルの設定
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# 日本語フォント設定
import matplotlib.font_manager as fm

def setup_japanese_font():
    """日本語フォントを設定する"""
    japanese_fonts = [
        'Hiragino Sans', 'Hiragino Maru Gothic Pro', 'AppleGothic',
        'Yu Gothic', 'MS Gothic',
        'Noto Sans CJK JP', 'IPAexGothic', 'TakaoPGothic',
    ]
    available_fonts = set(f.name for f in fm.fontManager.ttflist)
    for font in japanese_fonts:
        if font in available_fonts:
            plt.rcParams['font.family'] = font
            plt.rcParams['axes.unicode_minus'] = False
            return font
    return None

font_used = setup_japanese_font()
if font_used:
    print(f"日本語フォント: {font_used}")

np.random.seed(42)

print("✅ ライブラリのインポート完了")

---

## 1. 光の性質

### 1.1 電磁波としての光

光は**電磁波**の一種です。電磁波は波長によって分類されます：

| 波長 | 種類 |
|------|------|
| < 10nm | X線、ガンマ線 |
| 10-400nm | 紫外線 |
| **380-780nm** | **可視光** |
| 780nm-1mm | 赤外線 |
| > 1mm | 電波 |

カメラのセンサーは主に**可視光**（380-780nm）を検出します。

In [ ]:
# ============================================================
# 可視光スペクトルの可視化
# ============================================================

def wavelength_to_rgb(wavelength):
    """波長(nm)からRGB値に変換（近似）"""
    if wavelength < 380:
        return (0, 0, 0)
    elif wavelength < 440:
        r = -(wavelength - 440) / (440 - 380)
        g = 0
        b = 1
    elif wavelength < 490:
        r = 0
        g = (wavelength - 440) / (490 - 440)
        b = 1
    elif wavelength < 510:
        r = 0
        g = 1
        b = -(wavelength - 510) / (510 - 490)
    elif wavelength < 580:
        r = (wavelength - 510) / (580 - 510)
        g = 1
        b = 0
    elif wavelength < 645:
        r = 1
        g = -(wavelength - 645) / (645 - 580)
        b = 0
    elif wavelength <= 780:
        r = 1
        g = 0
        b = 0
    else:
        return (0, 0, 0)
    
    # 端での減衰
    if wavelength < 420:
        factor = 0.3 + 0.7 * (wavelength - 380) / (420 - 380)
    elif wavelength > 700:
        factor = 0.3 + 0.7 * (780 - wavelength) / (780 - 700)
    else:
        factor = 1.0
    
    return (r * factor, g * factor, b * factor)


fig, axes = plt.subplots(2, 1, figsize=(14, 6))

# 上: 可視光スペクトル
ax = axes[0]
wavelengths = np.linspace(380, 780, 400)
colors = [wavelength_to_rgb(w) for w in wavelengths]

for i, (w, c) in enumerate(zip(wavelengths, colors)):
    ax.axvline(w, color=c, linewidth=2)

ax.set_xlim(380, 780)
ax.set_xlabel('波長 (nm)', fontsize=12)
ax.set_title('可視光スペクトル (380-780nm)', fontsize=14, fontweight='bold')
ax.set_yticks([])

# 色の名前を追加
color_names = [
    (400, '紫'),
    (450, '青'),
    (500, '水色'),
    (550, '緑'),
    (580, '黄'),
    (620, '橙'),
    (700, '赤'),
]
for w, name in color_names:
    ax.annotate(name, (w, 0.5), ha='center', fontsize=11, fontweight='bold')

# 下: 電磁波スペクトル全体
ax = axes[1]
spectrum_data = [
    ('ガンマ線', -12, -10, 'purple'),
    ('X線', -10, -8, 'blue'),
    ('紫外線', -8, -6.4, 'violet'),
    ('可視光', -6.4, -6.1, 'green'),
    ('赤外線', -6.1, -3, 'red'),
    ('マイクロ波', -3, -1, 'orange'),
    ('電波', -1, 3, 'gray'),
]

for name, start, end, color in spectrum_data:
    ax.barh(0, end - start, left=start, height=0.5, color=color, alpha=0.7, edgecolor='black')
    ax.text((start + end) / 2, 0, name, ha='center', va='center', fontsize=10, fontweight='bold')

ax.set_xlim(-13, 4)
ax.set_xlabel('波長 (10^x メートル)', fontsize=12)
ax.set_title('電磁波スペクトル', fontsize=14, fontweight='bold')
ax.set_yticks([])
ax.axvline(-6.4, color='black', linestyle='--', alpha=0.5)
ax.axvline(-6.1, color='black', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

print("💡 可視光は電磁波スペクトルのごく一部（380-780nm）に過ぎません")

### 1.2 光の速度と屈折率

光の速度は媒質によって変化します：

$$c = \frac{c_0}{n}$$

- $c_0$: 真空中の光速（約 $3 \times 10^8$ m/s）
- $n$: 媒質の**屈折率**（refractive index）

| 媒質 | 屈折率 n |
|------|----------|
| 真空 | 1.000 |
| 空気 | 1.0003 |
| 水 | 1.333 |
| ガラス | 1.5-1.9 |
| ダイヤモンド | 2.42 |

---

## 2. 幾何光学の基本法則

### 2.1 光の直進性

均一な媒質中では、光は**直進**します。これが幾何光学の基礎です。

### 2.2 反射の法則

光が表面で反射するとき：

$$\theta_i = \theta_r$$

- $\theta_i$: 入射角（法線と入射光のなす角）
- $\theta_r$: 反射角（法線と反射光のなす角）

In [ ]:
# ============================================================
# 反射の法則の可視化
# ============================================================

fig, ax = plt.subplots(figsize=(10, 8))

# 反射面
ax.axhline(0, color='gray', linewidth=3)
ax.fill_between([-3, 3], [-0.5, -0.5], [0, 0], color='lightgray', alpha=0.5)

# 法線（破線）
ax.plot([0, 0], [0, 2.5], 'k--', linewidth=1.5, label='法線')

# 入射角
theta_i = np.radians(45)

# 入射光
x_in = -2 * np.sin(theta_i)
y_in = 2 * np.cos(theta_i)
ax.annotate('', xy=(0, 0), xytext=(x_in, y_in),
            arrowprops=dict(arrowstyle='->', color='blue', lw=2))
ax.text(x_in - 0.2, y_in + 0.1, '入射光', fontsize=12, color='blue')

# 反射光
x_out = 2 * np.sin(theta_i)
y_out = 2 * np.cos(theta_i)
ax.annotate('', xy=(x_out, y_out), xytext=(0, 0),
            arrowprops=dict(arrowstyle='->', color='red', lw=2))
ax.text(x_out + 0.1, y_out + 0.1, '反射光', fontsize=12, color='red')

# 角度の表示
arc1 = Arc((0, 0), 1, 1, angle=90, theta1=-45, theta2=0, color='blue', lw=2)
arc2 = Arc((0, 0), 1.2, 1.2, angle=90, theta1=0, theta2=45, color='red', lw=2)
ax.add_patch(arc1)
ax.add_patch(arc2)

ax.text(-0.5, 0.7, r'$\theta_i$', fontsize=14, color='blue')
ax.text(0.35, 0.7, r'$\theta_r$', fontsize=14, color='red')

# 式
ax.text(1.5, 2, r'反射の法則: $\theta_i = \theta_r$', fontsize=14, 
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

ax.set_xlim(-3, 3)
ax.set_ylim(-0.5, 3)
ax.set_aspect('equal')
ax.set_title('反射の法則', fontsize=14, fontweight='bold')
ax.axis('off')

plt.tight_layout()
plt.show()

### 2.3 スネルの法則（屈折の法則）

光が異なる媒質の境界を通過するとき：

$$n_1 \sin\theta_1 = n_2 \sin\theta_2$$

- $n_1, n_2$: それぞれの媒質の屈折率
- $\theta_1$: 入射角
- $\theta_2$: 屈折角

**重要**: 屈折率が高い媒質に入ると、光は法線に近づく方向に曲がります。

In [ ]:
# ============================================================
# スネルの法則の可視化
# ============================================================

def snell_law(n1, n2, theta1):
    """スネルの法則で屈折角を計算"""
    sin_theta2 = n1 * np.sin(theta1) / n2
    if abs(sin_theta2) > 1:
        return None  # 全反射
    return np.arcsin(sin_theta2)


fig, axes = plt.subplots(1, 3, figsize=(16, 6))

cases = [
    ('空気 → ガラス', 1.0, 1.5, 45),
    ('ガラス → 空気', 1.5, 1.0, 30),
    ('全反射 (臨界角超過)', 1.5, 1.0, 50),
]

for ax, (title, n1, n2, theta_deg) in zip(axes, cases):
    theta1 = np.radians(theta_deg)
    theta2 = snell_law(n1, n2, theta1)
    
    # 境界面
    ax.axhline(0, color='black', linewidth=2)
    
    # 媒質の表示
    ax.fill_between([-3, 3], [0, 0], [3, 3], color='lightblue', alpha=0.3)
    ax.fill_between([-3, 3], [-3, -3], [0, 0], color='lightyellow', alpha=0.5)
    ax.text(-2.5, 2.5, f'n₁ = {n1}', fontsize=12)
    ax.text(-2.5, -2.5, f'n₂ = {n2}', fontsize=12)
    
    # 法線
    ax.plot([0, 0], [-2.5, 2.5], 'k--', linewidth=1, alpha=0.5)
    
    # 入射光
    x_in = -2 * np.sin(theta1)
    y_in = 2 * np.cos(theta1)
    ax.annotate('', xy=(0, 0), xytext=(x_in, y_in),
                arrowprops=dict(arrowstyle='->', color='blue', lw=2))
    
    if theta2 is not None:
        # 屈折光
        x_out = 2 * np.sin(theta2)
        y_out = -2 * np.cos(theta2)
        ax.annotate('', xy=(x_out, y_out), xytext=(0, 0),
                    arrowprops=dict(arrowstyle='->', color='green', lw=2))
        ax.text(0.5, -1.5, f'θ₂ = {np.degrees(theta2):.1f}°', fontsize=11, color='green')
    else:
        # 全反射
        x_out = 2 * np.sin(theta1)
        y_out = 2 * np.cos(theta1)
        ax.annotate('', xy=(x_out, y_out), xytext=(0, 0),
                    arrowprops=dict(arrowstyle='->', color='red', lw=2))
        ax.text(0.5, 1.5, '全反射', fontsize=12, color='red', fontweight='bold')
    
    ax.text(-0.8, 1.2, f'θ₁ = {theta_deg}°', fontsize=11, color='blue')
    
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.set_aspect('equal')
    ax.set_title(title, fontsize=12, fontweight='bold')
    ax.axis('off')

plt.suptitle(r'スネルの法則: $n_1 \sin\theta_1 = n_2 \sin\theta_2$', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# 臨界角の計算
n1, n2 = 1.5, 1.0
critical_angle = np.degrees(np.arcsin(n2 / n1))
print(f"💡 ガラス→空気の臨界角: {critical_angle:.1f}°")
print(f"   入射角がこれを超えると全反射が起こります")

### 2.4 プリズムによる分散

屈折率は波長によってわずかに異なります（**分散**）。これにより、白色光はプリズムで虹色に分かれます。

$$n(\lambda) = A + \frac{B}{\lambda^2}$$

（コーシーの分散式の近似）

In [ ]:
# ============================================================
# プリズムによる分散の可視化
# ============================================================

fig, ax = plt.subplots(figsize=(12, 8))

# プリズムの形状
prism_vertices = np.array([
    [0, 0],
    [3, 0],
    [1.5, 2.6]  # 正三角形に近い形
])
prism = plt.Polygon(prism_vertices, fill=True, facecolor='lightcyan', 
                    edgecolor='black', linewidth=2, alpha=0.7)
ax.add_patch(prism)

# 入射光（白色光）
ax.annotate('', xy=(0.5, 1.3), xytext=(-2, 2.5),
            arrowprops=dict(arrowstyle='->', color='gray', lw=3))
ax.text(-2, 2.7, '白色光', fontsize=12, fontweight='bold')

# 分散後の光（虹色）
wavelengths = [400, 450, 500, 550, 600, 650, 700]
angles = np.linspace(0.3, -0.3, len(wavelengths))

for w, angle in zip(wavelengths, angles):
    color = wavelength_to_rgb(w)
    end_x = 2.5 + 3 * np.cos(angle - 0.3)
    end_y = 0.8 + 3 * np.sin(angle - 0.3)
    ax.annotate('', xy=(end_x, end_y), xytext=(2.5, 0.8),
                arrowprops=dict(arrowstyle='->', color=color, lw=2))

ax.text(5.5, 1.5, '赤', fontsize=11, color='red')
ax.text(5.5, 0.8, '緑', fontsize=11, color='green')
ax.text(5.5, 0.1, '青', fontsize=11, color='blue')
ax.text(5.5, -0.3, '紫', fontsize=11, color='purple')

# 説明
ax.text(0, -1, '短波長（紫・青）は長波長（赤）より大きく屈折', fontsize=11,
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

ax.set_xlim(-3, 7)
ax.set_ylim(-1.5, 3.5)
ax.set_aspect('equal')
ax.set_title('プリズムによる光の分散', fontsize=14, fontweight='bold')
ax.axis('off')

plt.tight_layout()
plt.show()

---

## 3. レンズの結像原理

### 3.1 なぜレンズが必要か？

ピンホール（小さな穴）でも像は作れますが、以下の問題があります：

1. **暗い**: 光量が極めて少ない
2. **回折**: 穴が小さすぎると回折によりボケる

レンズを使うことで、**多くの光を一点に集める**ことができます。

In [ ]:
# ============================================================
# ピンホール vs レンズの比較
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 左: ピンホールカメラ
ax = axes[0]

# 箱
ax.add_patch(plt.Rectangle((2, -2), 4, 4, fill=True, facecolor='lightgray', 
                            edgecolor='black', linewidth=2))

# ピンホール
ax.plot([2, 2], [-0.1, 0.1], 'k-', linewidth=8)
ax.plot([2, 2], [-0.05, 0.05], 'w-', linewidth=2)  # 穴

# 物体（矢印）
ax.annotate('', xy=(0, 1), xytext=(0, -1),
            arrowprops=dict(arrowstyle='<->', color='blue', lw=3))
ax.text(-0.5, 0, '物体', fontsize=11, color='blue')

# 光線
ax.plot([0, 2, 5], [1, 0, -0.8], 'r-', linewidth=1, alpha=0.7)
ax.plot([0, 2, 5], [-1, 0, 0.8], 'r-', linewidth=1, alpha=0.7)
ax.plot([0, 2, 5], [0, 0, 0], 'r-', linewidth=1, alpha=0.7)

# 像
ax.annotate('', xy=(5, 0.8), xytext=(5, -0.8),
            arrowprops=dict(arrowstyle='<->', color='green', lw=2))
ax.text(5.2, 0, '倒立像', fontsize=11, color='green')

ax.set_xlim(-1, 7)
ax.set_ylim(-2.5, 2.5)
ax.set_aspect('equal')
ax.set_title('ピンホールカメラ\n(シャープだが暗い)', fontsize=12, fontweight='bold')
ax.axis('off')

# 右: レンズカメラ
ax = axes[1]

# レンズ（楕円で表現）
lens = plt.matplotlib.patches.Ellipse((2, 0), 0.3, 2.5, fill=True, 
                                       facecolor='lightcyan', edgecolor='blue', linewidth=2)
ax.add_patch(lens)

# 物体
ax.annotate('', xy=(0, 1), xytext=(0, -1),
            arrowprops=dict(arrowstyle='<->', color='blue', lw=3))
ax.text(-0.5, 0, '物体', fontsize=11, color='blue')

# 光線（複数）
for y_start in [1, 0.5, 0, -0.5, -1]:
    # 物体の上端から
    ax.plot([0, 2], [1, y_start * 0.3], 'r-', linewidth=1, alpha=0.5)
    ax.plot([2, 5], [y_start * 0.3, -0.8], 'r-', linewidth=1, alpha=0.5)
    
    # 物体の下端から
    ax.plot([0, 2], [-1, y_start * 0.3], 'orange', linewidth=1, alpha=0.5)
    ax.plot([2, 5], [y_start * 0.3, 0.8], 'orange', linewidth=1, alpha=0.5)

# 像
ax.annotate('', xy=(5, 0.8), xytext=(5, -0.8),
            arrowprops=dict(arrowstyle='<->', color='green', lw=2))
ax.text(5.2, 0, '倒立像', fontsize=11, color='green')

ax.set_xlim(-1, 7)
ax.set_ylim(-2.5, 2.5)
ax.set_aspect('equal')
ax.set_title('レンズカメラ\n(多くの光を集光 → 明るい)', fontsize=12, fontweight='bold')
ax.axis('off')

plt.tight_layout()
plt.show()

print("💡 レンズは複数の光線を一点に集めることで、明るい像を形成します")

### 3.2 凸レンズと凹レンズ

| 種類 | 形状 | 性質 |
|------|------|------|
| 凸レンズ | 中央が厚い | 光を集める（収束）→ 実像を作る |
| 凹レンズ | 中央が薄い | 光を広げる（発散）→ 虚像を作る |

In [ ]:
# ============================================================
# 凸レンズと凹レンズの比較
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 凸レンズ
ax = axes[0]

# レンズの描画（2つの円弧で凸レンズを表現）
theta = np.linspace(-np.pi/3, np.pi/3, 100)
# 左側の曲面
x_left = -0.5 + 1.5 * np.cos(theta)
y_left = 1.5 * np.sin(theta)
# 右側の曲面
x_right = 0.5 - 1.5 * np.cos(theta)
y_right = 1.5 * np.sin(theta)

ax.fill_betweenx(y_left, x_left, x_right, color='lightcyan', alpha=0.7)
ax.plot(x_left, y_left, 'b-', linewidth=2)
ax.plot(x_right, y_right, 'b-', linewidth=2)

# 光軸
ax.axhline(0, color='gray', linestyle='--', linewidth=1)

# 平行光線 → 焦点に集まる
f = 2  # 焦点距離
for y in [0.8, 0.4, 0, -0.4, -0.8]:
    # 入射（平行）
    ax.plot([-3, 0], [y, y], 'r-', linewidth=1.5, alpha=0.7)
    # 屈折後（焦点へ）
    ax.plot([0, f], [y, 0], 'r-', linewidth=1.5, alpha=0.7)

# 焦点
ax.plot(f, 0, 'ko', markersize=10)
ax.text(f, -0.4, 'F (焦点)', fontsize=11, ha='center')

ax.set_xlim(-4, 4)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')
ax.set_title('凸レンズ（収束レンズ）', fontsize=14, fontweight='bold')
ax.axis('off')

# 凹レンズ
ax = axes[1]

# レンズの描画
theta = np.linspace(-np.pi/3, np.pi/3, 100)
x_left = 0.3 - 1.5 * np.cos(theta)
y_left = 1.5 * np.sin(theta)
x_right = -0.3 + 1.5 * np.cos(theta)
y_right = 1.5 * np.sin(theta)

ax.fill_betweenx(y_left, x_left, x_right, color='lightcyan', alpha=0.7)
ax.plot(x_left, y_left, 'b-', linewidth=2)
ax.plot(x_right, y_right, 'b-', linewidth=2)

# 光軸
ax.axhline(0, color='gray', linestyle='--', linewidth=1)

# 平行光線 → 発散（虚焦点から来たように見える）
f = -2  # 虚焦点
for y in [0.8, 0.4, 0, -0.4, -0.8]:
    # 入射（平行）
    ax.plot([-3, 0], [y, y], 'r-', linewidth=1.5, alpha=0.7)
    # 屈折後（発散）
    direction = np.array([1, -y/abs(f)])
    direction = direction / np.linalg.norm(direction) * 3
    ax.plot([0, direction[0]], [y, y + direction[1]], 'r-', linewidth=1.5, alpha=0.7)
    # 延長線（破線）→ 虚焦点へ
    ax.plot([0, f], [y, 0], 'r--', linewidth=1, alpha=0.4)

# 虚焦点
ax.plot(f, 0, 'ko', markersize=10, fillstyle='none', markeredgewidth=2)
ax.text(f, -0.4, 'F (虚焦点)', fontsize=11, ha='center')

ax.set_xlim(-4, 4)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')
ax.set_title('凹レンズ（発散レンズ）', fontsize=14, fontweight='bold')
ax.axis('off')

plt.tight_layout()
plt.show()

### 3.3 薄レンズの式

薄レンズ（厚みが無視できるレンズ）では、以下の関係が成り立ちます：

$$\frac{1}{f} = \frac{1}{d_o} + \frac{1}{d_i}$$

- $f$: 焦点距離（focal length）
- $d_o$: 物体距離（object distance）- レンズから物体までの距離
- $d_i$: 像距離（image distance）- レンズから像までの距離

**倍率**（Magnification）:

$$M = -\frac{d_i}{d_o} = \frac{h_i}{h_o}$$

- $h_o$: 物体の高さ
- $h_i$: 像の高さ
- 負の符号は像が倒立することを示す

In [ ]:
# ============================================================
# 薄レンズの結像シミュレーター
# ============================================================

def thin_lens_simulation(f, d_o, h_o=1.0):
    """
    薄レンズの結像をシミュレート
    
    Parameters:
    -----------
    f : float
        焦点距離
    d_o : float
        物体距離（正の値）
    h_o : float
        物体の高さ
    
    Returns:
    --------
    d_i : float
        像距離
    h_i : float
        像の高さ
    M : float
        倍率
    """
    # 薄レンズの式: 1/f = 1/d_o + 1/d_i
    d_i = 1 / (1/f - 1/d_o)
    
    # 倍率
    M = -d_i / d_o
    h_i = M * h_o
    
    return d_i, h_i, M


def plot_thin_lens(ax, f, d_o, h_o=1.0):
    """薄レンズの結像を可視化"""
    d_i, h_i, M = thin_lens_simulation(f, d_o, h_o)
    
    # レンズ
    ax.plot([0, 0], [-2, 2], 'b-', linewidth=3)
    ax.plot([-0.1, 0, 0.1], [1.8, 2, 1.8], 'b-', linewidth=2)
    ax.plot([-0.1, 0, 0.1], [-1.8, -2, -1.8], 'b-', linewidth=2)
    
    # 光軸
    ax.axhline(0, color='gray', linestyle='--', linewidth=1)
    
    # 焦点
    ax.plot(f, 0, 'ko', markersize=8)
    ax.plot(-f, 0, 'ko', markersize=8)
    ax.text(f, -0.3, 'F', fontsize=10, ha='center')
    ax.text(-f, -0.3, 'F', fontsize=10, ha='center')
    
    # 物体
    ax.annotate('', xy=(-d_o, h_o), xytext=(-d_o, 0),
                arrowprops=dict(arrowstyle='->', color='blue', lw=2))
    ax.text(-d_o - 0.3, h_o/2, '物体', fontsize=10, color='blue', va='center')
    
    # 主要光線を描画
    # 1. 光軸に平行 → 焦点を通る
    ax.plot([-d_o, 0], [h_o, h_o], 'r-', linewidth=1.5, alpha=0.7)
    ax.plot([0, max(d_i, f) + 1], [h_o, h_o - h_o * (max(d_i, f) + 1) / f], 'r-', linewidth=1.5, alpha=0.7)
    
    # 2. レンズ中心を通る（直進）
    ax.plot([-d_o, max(d_i, 0) + 1], [h_o, h_o * (1 - (max(d_i, 0) + 1 + d_o) / d_o)], 'g-', linewidth=1.5, alpha=0.7)
    
    # 3. 焦点を通る → 光軸に平行に出る
    ax.plot([-d_o, 0], [h_o, h_o * (1 - d_o / (d_o - f))], 'orange', linewidth=1.5, alpha=0.7)
    y_at_lens = h_o * (1 - d_o / (d_o - f))
    ax.plot([0, max(d_i, 0) + 1], [y_at_lens, y_at_lens], 'orange', linewidth=1.5, alpha=0.7)
    
    # 像
    if d_i > 0:  # 実像
        ax.annotate('', xy=(d_i, h_i), xytext=(d_i, 0),
                    arrowprops=dict(arrowstyle='->', color='green', lw=2))
        image_type = '実像（倒立）'
    else:  # 虚像
        ax.annotate('', xy=(d_i, h_i), xytext=(d_i, 0),
                    arrowprops=dict(arrowstyle='->', color='green', lw=2, linestyle='--'))
        image_type = '虚像（正立）'
    
    ax.text(d_i + 0.3, h_i/2, '像', fontsize=10, color='green', va='center')
    
    return d_i, h_i, M, image_type


# 異なる物体距離での比較
f = 2  # 焦点距離
object_distances = [6, 4, 3]  # 物体距離 > 2f, 2f > d_o > f, d_o > f

fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for ax, d_o in zip(axes, object_distances):
    d_i, h_i, M, image_type = plot_thin_lens(ax, f, d_o)
    
    ax.set_xlim(-8, 10)
    ax.set_ylim(-3, 3)
    ax.set_aspect('equal')
    ax.set_title(f'd_o = {d_o}, d_i = {d_i:.1f}\nM = {M:.2f}, {image_type}', 
                 fontsize=11, fontweight='bold')
    ax.axis('off')

plt.suptitle(f'薄レンズの結像 (f = {f})', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("💡 薄レンズの式: 1/f = 1/d_o + 1/d_i")
print("   物体が焦点距離の2倍より遠い → 縮小された実像")
print("   物体が焦点距離の2倍 → 等倍の実像")
print("   物体が焦点より遠く、2倍より近い → 拡大された実像")

In [ ]:
# ============================================================
# 薄レンズの式：インタラクティブな計算
# ============================================================

print("=" * 60)
print("薄レンズの式計算器")
print("=" * 60)

# 例題
examples = [
    (50, 200, "標準レンズ、物体が2m先"),
    (50, 100, "標準レンズ、物体が1m先"),
    (200, 500, "望遠レンズ、物体が5m先"),
    (24, 1000, "広角レンズ、物体が10m先"),
]

print(f"{'焦点距離':>10} {'物体距離':>10} {'像距離':>10} {'倍率':>10} {'説明'}")
print("-" * 70)

for f, d_o, description in examples:
    d_i, _, M = thin_lens_simulation(f, d_o)
    print(f"{f:>10.1f}mm {d_o:>10.1f}mm {d_i:>10.2f}mm {M:>10.4f} {description}")

print("\n💡 カメラでは像距離がほぼ焦点距離に等しくなります（物体が遠いため）")

---

## 4. カメラ光学パラメータ

### 4.1 焦点距離（Focal Length）

焦点距離 $f$ は、レンズの「ズーム力」を決定します：

- **短い焦点距離**（広角）: 広い範囲を写す、遠近感が強調される
- **長い焦点距離**（望遠）: 狭い範囲を拡大、遠近感が圧縮される

| 焦点距離 | 分類 | 用途 |
|----------|------|------|
| < 24mm | 超広角 | 建築、風景 |
| 24-35mm | 広角 | スナップ、風景 |
| 35-70mm | 標準 | ポートレート、スナップ |
| 70-200mm | 望遠 | スポーツ、ポートレート |
| > 200mm | 超望遠 | 野生動物、スポーツ |

In [ ]:
# ============================================================
# 焦点距離と画角の関係
# ============================================================

def calculate_fov(focal_length, sensor_size):
    """
    画角（Field of View）を計算
    
    FOV = 2 * arctan(sensor_size / (2 * focal_length))
    """
    return 2 * np.degrees(np.arctan(sensor_size / (2 * focal_length)))


# フルサイズセンサー (36mm x 24mm)
sensor_width = 36  # mm
sensor_height = 24  # mm

focal_lengths = [14, 24, 35, 50, 85, 135, 200, 400]

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 左: 画角のグラフ
ax = axes[0]
fl_range = np.linspace(10, 400, 100)
fov_horizontal = [calculate_fov(f, sensor_width) for f in fl_range]
fov_vertical = [calculate_fov(f, sensor_height) for f in fl_range]

ax.plot(fl_range, fov_horizontal, 'b-', linewidth=2, label='水平画角')
ax.plot(fl_range, fov_vertical, 'r-', linewidth=2, label='垂直画角')

# 代表的な焦点距離をマーク
for f in focal_lengths:
    fov_h = calculate_fov(f, sensor_width)
    ax.plot(f, fov_h, 'ko', markersize=8)
    ax.annotate(f'{f}mm\n({fov_h:.0f}°)', (f, fov_h + 5), ha='center', fontsize=9)

ax.set_xlabel('焦点距離 (mm)', fontsize=12)
ax.set_ylabel('画角 (度)', fontsize=12)
ax.set_title('焦点距離と画角の関係\n(フルサイズセンサー)', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xlim(0, 420)
ax.set_ylim(0, 120)

# 右: 画角の視覚的比較
ax = axes[1]

selected_fl = [24, 50, 135]
colors = ['red', 'green', 'blue']

for f, color in zip(selected_fl, colors):
    fov = np.radians(calculate_fov(f, sensor_width))
    
    # 扇形で画角を表示
    theta = np.linspace(-fov/2, fov/2, 100)
    r = 3
    x = np.concatenate([[0], r * np.sin(theta), [0]])
    y = np.concatenate([[0], r * np.cos(theta), [0]])
    
    ax.fill(x, y, color=color, alpha=0.3, label=f'{f}mm (FOV={np.degrees(fov):.0f}°)')
    ax.plot(x, y, color=color, linewidth=2)

ax.set_xlim(-4, 4)
ax.set_ylim(-0.5, 3.5)
ax.set_aspect('equal')
ax.set_title('画角の視覚的比較', fontsize=14, fontweight='bold')
ax.legend(loc='upper right')
ax.axis('off')

plt.tight_layout()
plt.show()

print("💡 画角の公式: FOV = 2 × arctan(センサーサイズ / (2 × 焦点距離))")

### 4.2 F値（絞り）

**F値**（f-number）は、レンズの開口の大きさを表します：

$$F = \frac{f}{D}$$

- $f$: 焦点距離
- $D$: 開口径（入射瞳径）

**F値の影響:**

| F値 | 開口 | 光量 | 被写界深度 |
|-----|------|------|------------|
| 小さい (f/1.4) | 大きい | 多い | 浅い（ボケが大きい）|
| 大きい (f/16) | 小さい | 少ない | 深い（全体にピント）|

In [ ]:
# ============================================================
# F値と開口の関係
# ============================================================

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 左: F値と開口径の関係
ax = axes[0]

f_numbers = [1.4, 2, 2.8, 4, 5.6, 8, 11, 16, 22]
focal_length = 50  # mm

for i, f_num in enumerate(f_numbers):
    diameter = focal_length / f_num
    circle = plt.Circle((i * 1.5, 0), diameter / 20, fill=False, 
                         edgecolor='blue', linewidth=2)
    ax.add_patch(circle)
    ax.text(i * 1.5, -1, f'f/{f_num}', ha='center', fontsize=10)
    ax.text(i * 1.5, 1, f'{diameter:.1f}mm', ha='center', fontsize=9, color='gray')

ax.set_xlim(-1, 13)
ax.set_ylim(-2, 2)
ax.set_aspect('equal')
ax.set_title(f'F値と開口径 (焦点距離 {focal_length}mm)', fontsize=14, fontweight='bold')
ax.axis('off')

# 右: 光量の比較
ax = axes[1]

# F値が√2倍になると光量は半分
f_numbers_plot = np.array([1.4, 2, 2.8, 4, 5.6, 8, 11, 16])
light_amount = 1 / (f_numbers_plot ** 2)
light_amount = light_amount / light_amount[0]  # f/1.4を基準に正規化

bars = ax.bar(range(len(f_numbers_plot)), light_amount, color='gold', edgecolor='black')
ax.set_xticks(range(len(f_numbers_plot)))
ax.set_xticklabels([f'f/{f}' for f in f_numbers_plot])
ax.set_ylabel('相対光量', fontsize=12)
ax.set_title('F値と光量の関係', fontsize=14, fontweight='bold')

# 光量の値を表示
for bar, light in zip(bars, light_amount):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
            f'{light:.2f}', ha='center', fontsize=9)

plt.tight_layout()
plt.show()

print("💡 F値が√2倍（1段）になると、光量は半分になります")
print("   f/1.4 → f/2 → f/2.8 → f/4 → f/5.6 → f/8 → f/11 → f/16")

### 4.3 被写界深度（Depth of Field, DoF）

**被写界深度**は、「ピントが合って見える」奥行きの範囲です。

$$\text{DoF} \approx \frac{2 u^2 N c}{f^2}$$

（近似式、物体が焦点距離より十分遠い場合）

- $u$: 物体距離
- $N$: F値
- $c$: 許容錯乱円径（Circle of Confusion）
- $f$: 焦点距離

**被写界深度に影響する要因:**

| 要因 | 被写界深度が深くなる条件 |
|------|------------------------|
| F値 | 大きいほど（絞るほど）深い |
| 焦点距離 | 短いほど深い |
| 物体距離 | 遠いほど深い |

In [ ]:
# ============================================================
# 被写界深度の計算と可視化
# ============================================================

def calculate_dof(focal_length, f_number, distance, coc=0.03):
    """
    被写界深度を計算
    
    Parameters:
    -----------
    focal_length : float
        焦点距離 (mm)
    f_number : float
        F値
    distance : float
        物体距離 (mm)
    coc : float
        許容錯乱円径 (mm)、フルサイズで約0.03mm
    
    Returns:
    --------
    near : float
        被写界深度の近点
    far : float
        被写界深度の遠点
    dof : float
        被写界深度
    """
    # 過焦点距離
    H = (focal_length ** 2) / (f_number * coc) + focal_length
    
    # 近点と遠点
    near = distance * (H - focal_length) / (H + distance - 2 * focal_length)
    
    if H > distance:
        far = distance * (H - focal_length) / (H - distance)
    else:
        far = float('inf')  # 無限遠までピント
    
    dof = far - near if far != float('inf') else float('inf')
    
    return near, far, dof


# F値による被写界深度の比較
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 左: F値の影響
ax = axes[0]

focal_length = 50  # mm
distance = 3000  # mm (3m)
f_numbers = [1.4, 2, 2.8, 4, 5.6, 8, 11, 16]

y_pos = np.arange(len(f_numbers))

for i, f_num in enumerate(f_numbers):
    near, far, dof = calculate_dof(focal_length, f_num, distance)
    
    # 被写界深度を棒グラフで表示
    if far != float('inf'):
        ax.barh(i, far - near, left=near, height=0.6, color='lightgreen', edgecolor='green')
        ax.text(far + 100, i, f'{dof/1000:.1f}m', va='center', fontsize=9)
    else:
        ax.barh(i, 5000, left=near, height=0.6, color='lightgreen', edgecolor='green')
        ax.text(near + 2500, i, '∞', va='center', fontsize=12)
    
    # ピント位置
    ax.axvline(distance, color='red', linestyle='--', alpha=0.5)

ax.set_yticks(y_pos)
ax.set_yticklabels([f'f/{f}' for f in f_numbers])
ax.set_xlabel('距離 (mm)', fontsize=12)
ax.set_title(f'F値と被写界深度\n(焦点距離{focal_length}mm、ピント距離{distance/1000}m)', 
             fontsize=12, fontweight='bold')
ax.axvline(distance, color='red', linestyle='--', label='ピント位置', alpha=0.7)
ax.legend()

# 右: 焦点距離の影響
ax = axes[1]

focal_lengths_compare = [24, 35, 50, 85, 135]
f_number = 2.8
distance = 3000  # mm

for i, fl in enumerate(focal_lengths_compare):
    near, far, dof = calculate_dof(fl, f_number, distance)
    
    if far != float('inf'):
        ax.barh(i, far - near, left=near, height=0.6, color='lightblue', edgecolor='blue')
        ax.text(far + 100, i, f'{dof/1000:.1f}m', va='center', fontsize=9)
    else:
        ax.barh(i, 8000 - near, left=near, height=0.6, color='lightblue', edgecolor='blue')
        ax.text(7000, i, '∞', va='center', fontsize=12)

ax.set_yticks(range(len(focal_lengths_compare)))
ax.set_yticklabels([f'{fl}mm' for fl in focal_lengths_compare])
ax.set_xlabel('距離 (mm)', fontsize=12)
ax.set_title(f'焦点距離と被写界深度\n(F値{f_number}、ピント距離{distance/1000}m)', 
             fontsize=12, fontweight='bold')
ax.axvline(distance, color='red', linestyle='--', label='ピント位置', alpha=0.7)
ax.legend()

plt.tight_layout()
plt.show()

print("💡 被写界深度のポイント:")
print("   - F値を大きくする（絞る）と被写界深度が深くなる")
print("   - 焦点距離が短いほど被写界深度が深い")
print("   - 物体が遠いほど被写界深度が深い")

---

## 5. 露出の三角形

写真の明るさ（露出）は、3つのパラメータで制御されます：

```
        シャッタースピード
              △
             / \
            /   \
           /     \
          /       \
         /_________\
     ISO感度      絞り(F値)
```

| パラメータ | 露出への影響 | 副作用 |
|------------|--------------|--------|
| シャッタースピード | 遅い→明るい | モーションブラー |
| F値（絞り） | 小さい→明るい | 被写界深度が浅い |
| ISO感度 | 高い→明るい | ノイズが増加 |

In [ ]:
# ============================================================
# 露出の三角形：トレードオフの可視化
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# シャッタースピード
ax = axes[0]
shutter_speeds = ['1/1000', '1/500', '1/250', '1/125', '1/60', '1/30', '1/15']
light = np.array([1, 2, 4, 8, 16, 32, 64])
blur = np.array([1, 2, 4, 8, 16, 32, 64])

ax.bar(range(len(shutter_speeds)), light, color='gold', alpha=0.7, label='光量')
ax2 = ax.twinx()
ax2.plot(range(len(shutter_speeds)), blur, 'r-o', linewidth=2, markersize=8, label='ブレのリスク')

ax.set_xticks(range(len(shutter_speeds)))
ax.set_xticklabels(shutter_speeds, rotation=45)
ax.set_ylabel('光量（相対）', fontsize=11, color='goldenrod')
ax2.set_ylabel('ブレのリスク（相対）', fontsize=11, color='red')
ax.set_title('シャッタースピード', fontsize=12, fontweight='bold')

# F値
ax = axes[1]
f_numbers = ['f/1.4', 'f/2', 'f/2.8', 'f/4', 'f/5.6', 'f/8', 'f/11']
light = np.array([16, 8, 4, 2, 1, 0.5, 0.25])
dof = np.array([1, 2, 4, 8, 16, 32, 64])

ax.bar(range(len(f_numbers)), light, color='gold', alpha=0.7, label='光量')
ax2 = ax.twinx()
ax2.plot(range(len(f_numbers)), dof, 'g-o', linewidth=2, markersize=8, label='被写界深度')

ax.set_xticks(range(len(f_numbers)))
ax.set_xticklabels(f_numbers, rotation=45)
ax.set_ylabel('光量（相対）', fontsize=11, color='goldenrod')
ax2.set_ylabel('被写界深度（相対）', fontsize=11, color='green')
ax.set_title('絞り (F値)', fontsize=12, fontweight='bold')

# ISO
ax = axes[2]
iso_values = ['100', '200', '400', '800', '1600', '3200', '6400']
sensitivity = np.array([1, 2, 4, 8, 16, 32, 64])
noise = np.array([1, 1.5, 2, 3, 5, 8, 15])

ax.bar(range(len(iso_values)), sensitivity, color='gold', alpha=0.7, label='感度')
ax2 = ax.twinx()
ax2.plot(range(len(iso_values)), noise, 'm-o', linewidth=2, markersize=8, label='ノイズ')

ax.set_xticks(range(len(iso_values)))
ax.set_xticklabels(iso_values, rotation=45)
ax.set_ylabel('感度（相対）', fontsize=11, color='goldenrod')
ax2.set_ylabel('ノイズレベル（相対）', fontsize=11, color='purple')
ax.set_title('ISO感度', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

print("💡 露出の三角形:")
print("   同じ明るさを得るために、3つのパラメータをトレードオフできます")
print("   例: シャッターを速くする → F値を下げるかISOを上げる")

---

## まとめ

### 🎯 このノートブックで学んだこと

**光の性質**
- ✓ 光は電磁波、可視光は380-780nm
- ✓ 屈折率により光速が変化

**幾何光学の法則**
- ✓ 反射の法則: $\theta_i = \theta_r$
- ✓ スネルの法則: $n_1 \sin\theta_1 = n_2 \sin\theta_2$

**レンズの結像**
- ✓ 薄レンズの式: $\frac{1}{f} = \frac{1}{d_o} + \frac{1}{d_i}$
- ✓ 倍率: $M = -\frac{d_i}{d_o}$

**カメラパラメータ**
- ✓ 焦点距離: 画角を決定
- ✓ F値: 光量と被写界深度のトレードオフ
- ✓ 被写界深度: ピントが合う範囲

### 📊 重要な数式まとめ

| 数式 | 意味 |
|------|------|
| $n_1 \sin\theta_1 = n_2 \sin\theta_2$ | スネルの法則 |
| $\frac{1}{f} = \frac{1}{d_o} + \frac{1}{d_i}$ | 薄レンズの式 |
| $F = \frac{f}{D}$ | F値の定義 |
| $\text{FOV} = 2 \arctan\left(\frac{W}{2f}\right)$ | 画角 |

---

## 🎓 自己評価クイズ

### Q1: 光が空気からガラス(n=1.5)に入射するとき、入射角が30°なら屈折角は？

<details>
<summary>💡 答えを見る</summary>

**答え**: 約19.5°

計算:
- $1.0 \times \sin(30°) = 1.5 \times \sin(\theta_2)$
- $\sin(\theta_2) = 0.5 / 1.5 = 0.333$
- $\theta_2 = \arcsin(0.333) \approx 19.5°$

</details>

---

### Q2: 焦点距離50mmのレンズで、物体が200mm先にあるとき、像距離は？

<details>
<summary>💡 答えを見る</summary>

**答え**: 約66.7mm

計算:
- $\frac{1}{50} = \frac{1}{200} + \frac{1}{d_i}$
- $\frac{1}{d_i} = \frac{1}{50} - \frac{1}{200} = \frac{4-1}{200} = \frac{3}{200}$
- $d_i = \frac{200}{3} \approx 66.7$mm

</details>

---

### Q3: F値を f/2.8 から f/5.6 に変更すると、光量は何倍になる？

<details>
<summary>💡 答えを見る</summary>

**答え**: 1/4倍（光量は4分の1に減少）

計算:
- f/2.8 → f/5.6 は2段分（各段で光量半分）
- または: 光量は$\frac{1}{F^2}$に比例
- $\frac{(1/5.6)^2}{(1/2.8)^2} = \frac{2.8^2}{5.6^2} = \frac{1}{4}$

</details>

---

### ✅ 学習チェックリスト

- [ ] スネルの法則を使って屈折角を計算できる
- [ ] 薄レンズの式で像距離を計算できる
- [ ] 焦点距離と画角の関係を説明できる
- [ ] F値が光量と被写界深度に与える影響を説明できる
- [ ] 露出の三角形の各要素のトレードオフを理解している

---

**次のステップ**: Notebook 51で、**ピンホールカメラと射影モデル**を学びます！

光学の知識を基に、3D空間の点を2D画像に変換する数学モデルを詳しく学びます。